# Solving a linear problem

Example of implementing a linear problem.
We'll use the biharmonic spline interpolation problem.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
def make_data(x):
    return (2*np.sin(x) - 2.5*np.sin(0.5*(x - 30)) + 3*np.cos(0.3*(x))
            + 0.5*np.sin(3*x))

In [ ]:
np.random.seed(5)
x_all = np.random.uniform(0, 21, 50)
y_all = make_data(x_all)
x_dense = np.linspace(x_all.min(), x_all.max(), 200)
y_dense = make_data(x_dense)
x_train = x_all[:-20]
x_test = x_all[-20:]
y_train = y_all[:-20]
y_test = y_all[-20:]

plt.figure()
plt.plot(x_train, y_train, 'x')
plt.plot(x_dense, y_dense, '-')

In [ ]:
from deeplook import LinearModel, LinearMisfit

In [ ]:
class BiharmonicSpline1D(LinearModel):
    
    def __init__(self, x_forces=None):
        self.x_forces = x_forces
        if x_forces is None:
            nparams = None
        else:
            nparams = x_forces.size
        super().__init__(nparams=nparams)
    
    def jacobian(self, x):
        ndata = x.size
        jac = np.empty((ndata, self.nparams), dtype=np.float32)
        for i in range(ndata):
            jac[i, :] = np.abs(x[i] - self.x_forces)**3
        return jac
    
    def predict(self, x):
        return self.jacobian(x).dot(self.params_)
    
    def fit(self, x, y):
        if self.x_forces is None:
            self.x_forces = x
            self.nparams = x.size
        jacobian = self.jacobian(x)
        self.misfit = LinearMisfit(data=y, jacobian=jacobian, normalize=True)
        self.params_ = self.misfit.minimize()
        return self

In [ ]:
np.average()

In [ ]:
interp = BiharmonicSpline1D().fit(x_train, y_train)
print(interp.r2_score(y_test, (x_test,)))

plt.figure()
plt.plot(x_train, y_train, 'x')
plt.plot(x_dense, interp.predict(x_dense), '-r')
plt.plot(x_dense, y_dense, '--b')

In [ ]:
n_forces = np.arange(3, x_train.size - 17, 1, dtype=np.int)
scores = np.zeros_like(n_forces, dtype=np.float)
folds = KFold()
for i, n in enumerate(n_forces):
    x_forces = np.linspace(x_train.min(), x_train.max(), n)
    score = 0
    candidate = BiharmonicSpline1D(x_forces)
    for train, test in folds.split(x_train):
        candidate.fit(x_train[train], y_train[train])
        score += candidate.r2_score(y_train[test], (x_train[test],)) 
    scores[i] = score/folds.n_splits
best_n = n_forces[np.argmax(scores)]
print(best_n)

In [ ]:
plt.figure()
plt.plot(n_forces, scores)

In [ ]:
interp_opt = BiharmonicSpline1D(np.linspace(x_train.min(), x_train.max(), best_n)).fit(x_train, y_train)
print(interp_opt.r2_score(y_test, (x_test,)))

plt.figure()
plt.plot(x_train, y_train, 'x')
plt.plot(x_dense, interp.predict(x_dense), '-r')
plt.plot(x_dense, interp_opt.predict(x_dense), '--g')
plt.plot(x_dense, y_dense, '--b')